In [1]:
import numpy as np
import pandas as pd 

samData = pd.read_csv("/qbio/nest/prom317/tutorial/transcriptomeAlignment/RPF_siLuc/ForwardStrand.out.sam", sep = '\t', names=['qname', 'flag', 'rname', 'pos', 'mapq', 'cigar', 'rnext', 'pnext', 'tlen', 'seq', 'qual','opt1', 'opt2', 'opt3', 'opt4'])
samData.head()

,qname,flag,rname,pos,mapq,cigar,rnext,pnext,tlen,seq,qual,opt1,opt2,opt3,opt4
0,SRR458756.2.803,0,ENSMUST00000240377.1|ENSMUSG00000119584.1|OTTM...,295,255,15M,*,0,0,TGGTGACTCTAGATA,B@@@@AAAAAAAAAA,NH:i:1,HI:i:1,AS:i:14,nM:i:0
1,SRR458756.2.918,0,ENSMUST00000091752.5|ENSMUSG00000069310.5|OTTM...,100,255,30M,*,0,0,CCAAGAATGGCTGGTACTAAGCAGACCGCT,A@A@@@A@@@?*.56A<2A<+;?*A??*?8,NH:i:1,HI:i:1,AS:i:27,nM:i:1
2,SRR458756.2.919,0,ENSMUST00000240377.1|ENSMUSG00000119584.1|OTTM...,815,255,28M,*,0,0,AAAAAATTAGAGTGTTCAAAGCAGGCCC,IIIIIIIHIHGGBG8AFFFFDGGBBGGG,NH:i:1,HI:i:1,AS:i:27,nM:i:0
3,SRR458756.2.922,0,ENSMUST00000240377.1|ENSMUSG00000119584.1|OTTM...,292,255,30M,*,0,0,CTTTGGCGACTCTAGATAACCTCGGGCCGA,##############################,NH:i:1,HI:i:1,AS:i:27,nM:i:1
4,SRR458756.2.923,0,ENSMUST00000023741.16|ENSMUSG00000048154.18|OT...,4117,3,1S17M,*,0,0,AGTGGCTGATATTGATAG,FC?BCGGGBBGEG@GDG>,NH:i:2,HI:i:1,AS:i:16,nM:i:0


In [2]:
samData['transcript_id'] = samData.rname.map(lambda p : p.split('|')[0])
samData['gene_id'] = samData.rname.map(lambda p : p.split('|')[1])
print(samData.shape)
samData.head()

(43076098, 17)


,qname,flag,rname,pos,mapq,cigar,rnext,pnext,tlen,seq,qual,opt1,opt2,opt3,opt4,transcript_id,gene_id
0,SRR458756.2.803,0,ENSMUST00000240377.1|ENSMUSG00000119584.1|OTTM...,295,255,15M,*,0,0,TGGTGACTCTAGATA,B@@@@AAAAAAAAAA,NH:i:1,HI:i:1,AS:i:14,nM:i:0,ENSMUST00000240377.1,ENSMUSG00000119584.1
1,SRR458756.2.918,0,ENSMUST00000091752.5|ENSMUSG00000069310.5|OTTM...,100,255,30M,*,0,0,CCAAGAATGGCTGGTACTAAGCAGACCGCT,A@A@@@A@@@?*.56A<2A<+;?*A??*?8,NH:i:1,HI:i:1,AS:i:27,nM:i:1,ENSMUST00000091752.5,ENSMUSG00000069310.5
2,SRR458756.2.919,0,ENSMUST00000240377.1|ENSMUSG00000119584.1|OTTM...,815,255,28M,*,0,0,AAAAAATTAGAGTGTTCAAAGCAGGCCC,IIIIIIIHIHGGBG8AFFFFDGGBBGGG,NH:i:1,HI:i:1,AS:i:27,nM:i:0,ENSMUST00000240377.1,ENSMUSG00000119584.1
3,SRR458756.2.922,0,ENSMUST00000240377.1|ENSMUSG00000119584.1|OTTM...,292,255,30M,*,0,0,CTTTGGCGACTCTAGATAACCTCGGGCCGA,##############################,NH:i:1,HI:i:1,AS:i:27,nM:i:1,ENSMUST00000240377.1,ENSMUSG00000119584.1
4,SRR458756.2.923,0,ENSMUST00000023741.16|ENSMUSG00000048154.18|OT...,4117,3,1S17M,*,0,0,AGTGGCTGATATTGATAG,FC?BCGGGBBGEG@GDG>,NH:i:2,HI:i:1,AS:i:16,nM:i:0,ENSMUST00000023741.16,ENSMUSG00000048154.18


In [3]:
gtfData = pd.read_table("/qbio/nest/prom317/tutorial/references/GRCm39.genome.gtf", sep = '\t', names = ["seq_name","source","feature","start","end","score","strand","frame","attribute"])
gtfData.iloc[5:10]

,seq_name,source,feature,start,end,score,strand,frame,attribute
5,chr1,HAVANA,gene,3143476.0,3144545.0,.,+,.,"gene_id ""ENSMUSG00000102693.2""; gene_type ""TEC..."
6,chr1,HAVANA,transcript,3143476.0,3144545.0,.,+,.,"gene_id ""ENSMUSG00000102693.2""; transcript_id ..."
7,chr1,HAVANA,exon,3143476.0,3144545.0,.,+,.,"gene_id ""ENSMUSG00000102693.2""; transcript_id ..."
8,chr1,ENSEMBL,gene,3172239.0,3172348.0,.,+,.,"gene_id ""ENSMUSG00000064842.3""; gene_type ""snR..."
9,chr1,ENSEMBL,transcript,3172239.0,3172348.0,.,+,.,"gene_id ""ENSMUSG00000064842.3""; transcript_id ..."


In [4]:
samData['start_pos'] = np.nan
samData['end_pos'] = np.nan
samData.head()

,qname,flag,rname,pos,mapq,cigar,rnext,pnext,tlen,seq,qual,opt1,opt2,opt3,opt4,transcript_id,gene_id,start_pos,end_pos
0,SRR458756.2.803,0,ENSMUST00000240377.1|ENSMUSG00000119584.1|OTTM...,295,255,15M,*,0,0,TGGTGACTCTAGATA,B@@@@AAAAAAAAAA,NH:i:1,HI:i:1,AS:i:14,nM:i:0,ENSMUST00000240377.1,ENSMUSG00000119584.1,NaN,NaN
1,SRR458756.2.918,0,ENSMUST00000091752.5|ENSMUSG00000069310.5|OTTM...,100,255,30M,*,0,0,CCAAGAATGGCTGGTACTAAGCAGACCGCT,A@A@@@A@@@?*.56A<2A<+;?*A??*?8,NH:i:1,HI:i:1,AS:i:27,nM:i:1,ENSMUST00000091752.5,ENSMUSG00000069310.5,NaN,NaN
2,SRR458756.2.919,0,ENSMUST00000240377.1|ENSMUSG00000119584.1|OTTM...,815,255,28M,*,0,0,AAAAAATTAGAGTGTTCAAAGCAGGCCC,IIIIIIIHIHGGBG8AFFFFDGGBBGGG,NH:i:1,HI:i:1,AS:i:27,nM:i:0,ENSMUST00000240377.1,ENSMUSG00000119584.1,NaN,NaN
3,SRR458756.2.922,0,ENSMUST00000240377.1|ENSMUSG00000119584.1|OTTM...,292,255,30M,*,0,0,CTTTGGCGACTCTAGATAACCTCGGGCCGA,##############################,NH:i:1,HI:i:1,AS:i:27,nM:i:1,ENSMUST00000240377.1,ENSMUSG00000119584.1,NaN,NaN
4,SRR458756.2.923,0,ENSMUST00000023741.16|ENSMUSG00000048154.18|OT...,4117,3,1S17M,*,0,0,AGTGGCTGATATTGATAG,FC?BCGGGBBGEG@GDG>,NH:i:2,HI:i:1,AS:i:16,nM:i:0,ENSMUST00000023741.16,ENSMUSG00000048154.18,NaN,NaN


In [5]:
gtfPlusData = gtfData.loc[gtfData.strand == "+"]
del gtfData
gtfPlusData['gene_id'] = gtfPlusData.attribute.map(lambda p: p.split(';')[0].split(' ')[1][1:-1])
gtfPlusData.gene_id.value_counts()

ENSMUSG00000052613.17    2003
ENSMUSG00000069601.15    1772
ENSMUSG00000021843.18    1695
ENSMUSG00000064302.14    1411
ENSMUSG00000033788.16    1411
                         ... 
ENSMUSG00000119588.1        3
ENSMUSG00000117051.2        3
ENSMUSG00000107178.2        3
ENSMUSG00000070610.5        3
ENSMUSG00000117980.2        3
Name: gene_id, Length: 26876, dtype: int64

In [6]:
transcript_list = samData['transcript_id'].unique()
transDict = {}
for ts in transcript_list:
    transDict[ts] = [[],[]]
print(transcript_list)

['ENSMUST00000240377.1' 'ENSMUST00000091752.5' 'ENSMUST00000023741.16' ...
 'ENSMUST00000141667.2' 'ENSMUST00000099986.3' 'ENSMUST00000225350.2']


In [7]:
gtfExonData = gtfPlusData.loc[gtfPlusData.feature == 'exon']
gtfExonData["transcript_id"] = gtfExonData.iloc[:,8].map(lambda p: p.split(';')[1].split(' ')[2][1:-1])
gtfStartData = gtfPlusData.loc[gtfPlusData.feature == 'start_codon']
gtfStartData["transcript_id"] = gtfStartData.iloc[:,8].map(lambda p: p.split(';')[1].split(' ')[2][1:-1])
gtfStopData = gtfPlusData.loc[gtfPlusData.feature == 'stop_codon']
gtfStopData["transcript_id"] = gtfStopData.iloc[:,8].map(lambda p: p.split(';')[1].split(' ')[2][1:-1])
del gtfPlusData
gtfStartData.head()

/blaze/prom317/conda/envs/lab/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/blaze/prom317/conda/envs/lab/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/blaze/prom317/conda/envs/lab/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

,seq_name,source,feature,start,end,score,strand,frame,attribute,gene_id,transcript_id
361,chr1,HAVANA,start_codon,4878137.0,4878139.0,.,+,0,"gene_id ""ENSMUSG00000025903.15""; transcript_id...",ENSMUSG00000025903.15,ENSMUST00000134384.8
386,chr1,HAVANA,start_codon,4878137.0,4878139.0,.,+,0,"gene_id ""ENSMUSG00000025903.15""; transcript_id...",ENSMUSG00000025903.15,ENSMUST00000027036.11
409,chr1,HAVANA,start_codon,4878137.0,4878139.0,.,+,0,"gene_id ""ENSMUSG00000025903.15""; transcript_id...",ENSMUSG00000025903.15,ENSMUST00000150971.8
430,chr1,HAVANA,start_codon,4878137.0,4878139.0,.,+,0,"gene_id ""ENSMUSG00000025903.15""; transcript_id...",ENSMUSG00000025903.15,ENSMUST00000119612.9
449,chr1,HAVANA,start_codon,4878137.0,4878139.0,.,+,0,"gene_id ""ENSMUSG00000025903.15""; transcript_id...",ENSMUSG00000025903.15,ENSMUST00000137887.8


In [8]:
count = 0
for ts in transDict.keys():
    count += 1
    if count % 10000 == 0:
        print(count)
    tsGtfExon = gtfExonData.loc[gtfExonData.transcript_id == ts]
    tsGtfStart = gtfStartData.loc[gtfStartData.transcript_id == ts]
    tsGtfStop = gtfStopData.loc[gtfStopData.transcript_id == ts]
    for idx, row in tsGtfExon.iterrows():
        transDict[ts][0].append(row['start'])
        transDict[ts][1].append(row['end'])
    if not (tsGtfStart.empty):
        transDict[ts].append(['start_codon',tsGtfStart.iloc[0,3]] )
    if not (tsGtfStop.empty):
        transDict[ts].append(['stop_codon',tsGtfStop.iloc[0,3]] )
    

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000


In [9]:
codingTransDict = {}
for i in transDict.keys():
    if len(transDict[i]) == 4:
        codingTransDict[i] = transDict[i]
print(len(codingTransDict))

21605


In [10]:
for i in codingTransDict.keys():
    exonStart = codingTransDict[i][0]
    exonEnd = codingTransDict[i][1]
    startCod = codingTransDict[i][2][1]
    stopCod = codingTransDict[i][3][1]
    exonLength = 0
    startPos = 0
    stopPos = 0
    for j in range(len(exonStart)):
        if exonStart[j] <= startCod < exonEnd[j]:
            startPos = exonLength + startCod - exonStart[j] + 1
            exonLength = exonLength + exonEnd[j] - exonStart[j] + 1
        else:
            exonLength = exonLength + exonEnd[j] - exonStart[j] + 1
    exonLength = 0
    for j in range(len(exonStart)):
        if exonStart[j] <= stopCod < exonEnd[j]:
            stopPos = exonLength + stopCod - exonStart[j] + 1
            exonLength = exonLength + exonEnd[j] - exonStart[j] + 1
        else:
            exonLength = exonLength + exonEnd[j] - exonStart[j] + 1
    codingTransDict[i].append(exonLength)
    codingTransDict[i].append(startPos)
    codingTransDict[i].append(stopPos)

In [12]:
codingDF = pd.DataFrame.from_dict(codingTransDict, orient = 'index')
codingDF = codingDF.drop(columns=[0,1,2,3,4])
codingDF.to_csv('CodingTranscript.csv', sep='\t')